In [9]:
!pip install -qU kaggle

In [10]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/


In [11]:
!chmod 600 ~/.kaggle/kaggle.json


In [12]:
!kaggle datasets download ryanrudes/yttts-speech

100% 9.91G/9.91G [08:00<00:00, 21.4MB/s]
100% 9.91G/9.91G [08:00<00:00, 22.2MB/s]


In [13]:
!unzip yttts-speech.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/yexR53My2O4/00:03:26,459-00:03:28,290/audio.wav  
  inflating: data/yexR53My2O4/00:03:26,459-00:03:28,290/subtitles.txt  
  inflating: data/yexR53My2O4/00:03:28,290-00:03:30,030/audio.wav  
  inflating: data/yexR53My2O4/00:03:28,290-00:03:30,030/subtitles.txt  
  inflating: data/yexR53My2O4/00:03:30,030-00:03:33,300/audio.wav  
  inflating: data/yexR53My2O4/00:03:30,030-00:03:33,300/subtitles.txt  
  inflating: data/yexR53My2O4/00:03:33,300-00:03:36,510/audio.wav  
  inflating: data/yexR53My2O4/00:03:33,300-00:03:36,510/subtitles.txt  
  inflating: data/yexR53My2O4/00:03:36,510-00:03:39,150/audio.wav  
  inflating: data/yexR53My2O4/00:03:36,510-00:03:39,150/subtitles.txt  
  inflating: data/yexR53My2O4/00:03:39,150-00:03:41,780/audio.wav  
  inflating: data/yexR53My2O4/00:03:39,150-00:03:41,780/subtitles.txt  
  inflating: data/yexR53My2O4/00:03:41,780-00:03:45,660/audio.wav  
  inflating: data/yexR53My2O4/00:03:41,78

In [14]:
!pip install -qU bs4 tqdm datasets

In [15]:
import os
import time
import requests
from tqdm.auto import tqdm
from bs4 import BeautifulSoup

In [16]:
video_ids = os.listdir("data")
video_ids[:5]

['YqvhDPd1UEw', '2lkUNDZld-4', 'BnpB3GrpsfM', 'oDKXwxaGkNA', 'YPfUiOMYOEE']

In [17]:
splits = sorted(os.listdir(f"data/{video_ids[0]}"))
splits[:5]

['00:00:00,530-00:00:04,350',
 '00:00:04,350-00:00:07,470',
 '00:00:07,470-00:00:09,809',
 '00:00:09,809-00:00:13,559',
 '00:00:13,559-00:00:15,089']

In [18]:
with open(f"data/{video_ids[0]}/{splits[0]}/subtitles.txt") as f:
    text = f.read()
print(text)

hi everyone welcome to lecture 12 of


In [19]:
documents = []
for video_id in tqdm(video_ids):
    splits = sorted(os.listdir(f"data/{video_id}"))
    # we start at 00:00:00
    start_timestamp = "00:00:00"
    passage = ""
    for i, s in enumerate(splits):
        with open(f"data/{video_id}/{s}/subtitles.txt") as f:
            # append tect to current chunk
            out = f.read()
            passage += " " + out
        # average sentence length is 75-100 characters so we will cut off
        # around 3-4 sentences
        if len(passage) > 360:
            # now we've hit the needed length create a record
            # extract the end timestamp from the filename
            end_timestamp = s.split("-")[1].split(",")[0]
            # extract string timestamps to actual datetime objects
            start = time.strptime(start_timestamp,"%H:%M:%S")
            end = time.strptime(end_timestamp,"%H:%M:%S")
            # now we extract the second/minute/hour values and convert
            # to total number of seconds
            start_second = start.tm_sec + start.tm_min*60 + start.tm_hour*3600
            end_second = end.tm_sec + end.tm_min*60 + end.tm_hour*3600
            # save this to the documents list
            documents.append({
                "video_id": video_id,
                "text": passage,
                "start_second": start_second,
                "end_second": end_second,
                "url": f"https://www.youtube.com/watch?v={video_id}&t={start_second}s",
            })
            # now we update the start_timestamp for the next chunk
            start_timestamp = end_timestamp
            # refresh passage
            passage = ""

  0%|          | 0/127 [00:00<?, ?it/s]

In [20]:
documents[:3]

[{'video_id': 'YqvhDPd1UEw',
  'text': " hi everyone welcome to lecture 12 of deep unsupervised learning spring 2020 today we'll cover representation learning in reinforcement learning first before I start I wanna give a big thank you to many colleagues and friends who have contributed to this lecture through sharing their insights illustrations slides and videos that all very directly contributed to well I'll be sharing in",
  'start_second': 0,
  'end_second': 28,
  'url': 'https://www.youtube.com/watch?v=YqvhDPd1UEw&t=0s'},
 {'video_id': 'YqvhDPd1UEw',
  'text': " this lecture here today thank you so Terry what class has been about unsupervised learning today we're actually going to look at how one surprise learning and reinforcement can be brought together in a way that inspires learning to make reinforced learning more efficient but we haven't covered reinforcement learning yet in this class and so what I'm first gonna",
  'start_second': 28,
  'end_second': 52,
  'url': 'https://

In [21]:
import lxml  # if on mac, pip/conda install lxml

metadata = {}
for _id in tqdm(video_ids):
    r = requests.get(f"https://www.youtube.com/watch?v={_id}")
    soup = BeautifulSoup(r.content, 'lxml')  # lxml package is used here
    try:
        title = soup.find("meta", property="og:title").get("content")
        thumbnail = soup.find("meta", property="og:image").get("content")
        metadata[_id] = {"title": title, "thumbnail": thumbnail}
    except Exception as e:
        print(e)
        print(_id)
        metadata[_id] = {"title": "", "thumbnail": ""}

len(metadata)

  0%|          | 0/127 [00:00<?, ?it/s]

'NoneType' object has no attribute 'get'
arbbhHyRP90
'NoneType' object has no attribute 'get'
bM9BXu9lPZ0
'NoneType' object has no attribute 'get'
fpDaQxG5w4o
'NoneType' object has no attribute 'get'
xtOg44r6dsE


127

In [22]:
metadata['ZPewmEu7644']

{'title': 'GANS for Semi-Supervised Learning in Keras (7.4)',
 'thumbnail': 'https://i.ytimg.com/vi/ZPewmEu7644/maxresdefault.jpg'}

In [23]:
for i, doc in enumerate(documents):
    _id = doc['video_id']
    meta = metadata[_id]
    # add metadata to existing doc
    documents[i] = {**doc, **meta}

In [24]:
documents[0]

{'video_id': 'YqvhDPd1UEw',
 'text': " hi everyone welcome to lecture 12 of deep unsupervised learning spring 2020 today we'll cover representation learning in reinforcement learning first before I start I wanna give a big thank you to many colleagues and friends who have contributed to this lecture through sharing their insights illustrations slides and videos that all very directly contributed to well I'll be sharing in",
 'start_second': 0,
 'end_second': 28,
 'url': 'https://www.youtube.com/watch?v=YqvhDPd1UEw&t=0s',
 'title': 'L12 Representation Learning for Reinforcement Learning --- CS294-158 UC Berkeley Spring 2020',
 'thumbnail': 'https://i.ytimg.com/vi/YqvhDPd1UEw/maxresdefault.jpg'}

In [25]:
import json

with open("train.jsonl", "w") as f:
    for doc in documents:
        json.dump(doc, f)
        f.write('\n')

In [26]:
with open("train.jsonl") as f:
    d = f.readlines()

In [27]:
d[:3]

['{"video_id": "YqvhDPd1UEw", "text": " hi everyone welcome to lecture 12 of deep unsupervised learning spring 2020 today we\'ll cover representation learning in reinforcement learning first before I start I wanna give a big thank you to many colleagues and friends who have contributed to this lecture through sharing their insights illustrations slides and videos that all very directly contributed to well I\'ll be sharing in", "start_second": 0, "end_second": 28, "url": "https://www.youtube.com/watch?v=YqvhDPd1UEw&t=0s", "title": "L12 Representation Learning for Reinforcement Learning --- CS294-158 UC Berkeley Spring 2020", "thumbnail": "https://i.ytimg.com/vi/YqvhDPd1UEw/maxresdefault.jpg"}\n',
 '{"video_id": "YqvhDPd1UEw", "text": " this lecture here today thank you so Terry what class has been about unsupervised learning today we\'re actually going to look at how one surprise learning and reinforcement can be brought together in a way that inspires learning to make reinforced learni

In [28]:
!pip install -U datasets sentence-transformers pinecone-client tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 97.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=0a4e64126561968bdbe965f1b5d53bca033cda225af0840efa0b08e429e7e9f2
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-trans

In [29]:
from datasets import load_dataset

yt = load_dataset(
    'pinecone/yt-transcriptions',
    split = 'train',
    revision = '926a45'

)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/pinecone___json/pinecone--yt-transcriptions-c0a57d2f8951521b/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


In [30]:
for x in yt:
  print(x)
  break

{'video_id': 'ZPewmEu7644', 'text': " hi this is Jeff Dean welcome to applications of deep neural networks of Washington University in this video we're going to look at how we can use ganz to generate additional training data for the latest on my a I course and projects click subscribe in the bell next to it to be notified of every new video Dan's have a wide array of uses beyond just the face generation that you", 'start_second': 0, 'end_second': 20, 'url': 'https://www.youtube.com/watch?v=ZPewmEu7644&t=0s', 'title': 'GANS for Semi-Supervised Learning in Keras (7.4)', 'thumbnail': 'https://i.ytimg.com/vi/ZPewmEu7644/maxresdefault.jpg'}


In [31]:
from sentence_transformers import SentenceTransformer
import torch

if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'

retriever = SentenceTransformer('flax-sentence-embeddings/all_datasets_v3_mpnet-base')

retriever.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [32]:
embed_dim = retriever.get_sentence_embedding_dimension()

In [33]:
embed_dim

768

In [36]:
import pinecone

pinecone.init(
    api_key="05d04c60-f94b-4e56-9d98-3dcbe5cbf914",
    environment="us-west4-gcp-free"
)

# create index
pinecone.create_index(
    "youtube-search",
    dimension=embed_dim,
    metric="cosine"
)

# connect to new index
index = pinecone.Index("youtube-search")

In [37]:
from tqdm.auto import tqdm
docs = []
batch_size = 64

for i in tqdm(range(0,len(yt),batch_size)):

  i_end = min(i+batch_size,len(yt))
  batch = yt[i:i_end]                          # extract batch from YT transactions
  embeds = retriever.encode(batch['text']).tolist()   # encode a batch
  ids = [f"{x[0]}-{x[1]}" for x in zip(batch['video_id'], batch['start_second'])]

  meta = [{
      'video_id': x[0],
      'title': x[1],
      'text': x[2],
      'start_second': x[3],
      'end_second': x[4],
      'url': x[5],
      'thumbnail': x[6]
  } for x in zip(
      batch['video_id'],
      batch['title'],
      batch['text'],
      batch['start_second'],
      batch['end_second'],
      batch['url'],
      batch['thumbnail']
  )]

  to_upsert = list(zip(ids,embeds,meta))   # create list of (IDs, vectors, metadata) to upsert
  index.upsert(vectors = to_upsert)        # adding to pinecone

  0%|          | 0/177 [00:00<?, ?it/s]

In [38]:
index.describe_index_stats

<bound method Index.describe_index_stats of <pinecone.index.Index object at 0x7fda702548b0>>

# Querying

In [50]:
query = 'what is principal component analysis?'
xq = retriever.encode([query]).tolist()

In [51]:
xc = index.query(xq, top_k = 3, include_metadata = True)
for context in xc['matches']:
  print(context['metadata']['text'], end = '\n')
  print('url',context['metadata']['url'])

 is something red concepts in the same places so this is a PCA and then blue I think these blue is the attention codes for identify the red things and in red or the generation code for make something red and they will be put in the same place which is pretty cool it means that the energy function really learns the feature of something being red I find this pretty pretty neat and then
url https://www.youtube.com/watch?v=Cs_j-oNwGgg&t=1933s
 in PCA the the the region of the space that is perfectly reconstructed is the principal subspace right in this case here if the data points are sampled from this from the sparrow the principal subspace here dimension 1 is this so this has become our reconstruction error 0 and everything else grows could racket quadratically right because you take a point you project it on this and so if
url https://www.youtube.com/watch?v=A7AnCvYDQrU&t=1922s
 think of CPC is trying to do something like multi-step prediction tasks so if you look at these multiple pred

In [43]:
xc

{'matches': [{'id': 'NTz4rJS9BAI-140',
              'metadata': {'end_second': 158.0,
                           'start_second': 140.0,
                           'text': ' upset some of my colleagues nobly Mike '
                                   'Jordan but I assess on my college might '
                                   "say this is all machine learning is it's "
                                   'just predict making predictions from '
                                   'examples I have a set of something I have '
                                   'a set of something else and I want to find '
                                   'prediction functions that map X to Y I '
                                   "know there's more than that but this is "
                                   'the backbone this is why all of you are '
                                   "here this is why they're",
                           'thumbnail': 'https://i.ytimg.com/vi/NTz4rJS9BAI/maxresdefault.jpg',
